In [42]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [80]:
label_lg = "/Users/dutking/LOCAL/AI_uni/radlogix/dataset/labels/LUNG1-001/LUNG1-001_effusion_first_reviewer.nii.gz"
feature_lg = "/Users/dutking/LOCAL/AI_uni/radlogix/dataset/features/LUNG1-001/09-18-2008-StudyID-NA-69331/0.000000-NA-82046"

label_sm = "/Users/dutking/LOCAL/AI_uni/radlogix/dataset/effusions_052023/LUNG1-285/LUNG1285_effusion_second_reviewer.nii.gz"
feature_sm = "/Users/dutking/LOCAL/AI_uni/radlogix/dataset/features/LUNG1-285/08-03-2008-StudyID-NA-05059/0.000000-NA-50027"

TARGET_SLICES = 192
DIMENSION = 256
PAD_FILLER = -1024

In [43]:
v = sitk.ImageViewer()
v.SetFileExtension(".nii")
v.SetApplication("/Applications/Fiji.app/Contents/MacOS/ImageJ-macosx")

In [44]:
def get_feature(dicom_folder):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(dicom_folder)
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    image = sitk.DICOMOrient(image, "LPS")
    return image


def get_label(file):
    image = sitk.ReadImage(file)
    image = sitk.DICOMOrient(image, "LPS")
    return image

In [45]:
def resample_spacing(image):
    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator(sitk.sitkLinear)
    resample.SetOutputDirection(image.GetDirection())
    resample.SetOutputOrigin(image.GetOrigin())
    new_spacing = [1, 1, 1]
    resample.SetOutputSpacing(new_spacing)

    orig_size = np.array(image.GetSize(), dtype=np.int32)
    orig_spacing = image.GetSpacing()
    new_size = orig_size * (np.array(orig_spacing) / np.array(new_spacing))
    new_size = np.ceil(new_size).astype(np.int32)  #  Image dimensions are in integers
    new_size = [int(s) for s in new_size]
    resample.SetSize(new_size)

    new_image = resample.Execute(image)
    return new_image

In [46]:
def resample_size(patient_CT, resize_factor):
    # original_CT = sitk.ReadImage(patient_CT,sitk.sitkInt32)
    original_CT = patient_CT
    dimension = original_CT.GetDimension()
    reference_physical_size = np.zeros(original_CT.GetDimension())
    reference_physical_size[:] = [
        (sz - 1) * spc if sz * spc > mx else mx
        for sz, spc, mx in zip(
            original_CT.GetSize(), original_CT.GetSpacing(), reference_physical_size
        )
    ]

    reference_origin = original_CT.GetOrigin()
    reference_direction = original_CT.GetDirection()

    reference_size = [round(sz / resize_factor) for sz in original_CT.GetSize()]
    reference_spacing = [
        phys_sz / (sz - 1)
        for sz, phys_sz in zip(reference_size, reference_physical_size)
    ]

    reference_image = sitk.Image(reference_size, original_CT.GetPixelIDValue())
    reference_image.SetOrigin(reference_origin)
    reference_image.SetSpacing(reference_spacing)
    reference_image.SetDirection(reference_direction)

    reference_center = np.array(
        reference_image.TransformContinuousIndexToPhysicalPoint(
            np.array(reference_image.GetSize()) / 2.0
        )
    )

    transform = sitk.AffineTransform(dimension)
    transform.SetMatrix(original_CT.GetDirection())

    transform.SetTranslation(np.array(original_CT.GetOrigin()) - reference_origin)

    centering_transform = sitk.TranslationTransform(dimension)
    img_center = np.array(
        original_CT.TransformContinuousIndexToPhysicalPoint(
            np.array(original_CT.GetSize()) / 2.0
        )
    )
    centering_transform.SetOffset(
        np.array(transform.GetInverse().TransformPoint(img_center) - reference_center)
    )
    centered_transform = sitk.CompositeTransform(transform)
    centered_transform.AddTransform(centering_transform)

    # sitk.Show(sitk.Resample(original_CT, reference_image, centered_transform, sitk.sitkLinear, 0.0))

    return sitk.Resample(
        original_CT, reference_image, centered_transform, sitk.sitkLinear, 0.0
    )

In [47]:
def plot_overlay(feature, label):
    mid_slice = int(feature.shape[0] / 2)
    print(mid_slice)
    feature_slice = feature[mid_slice, :, :]
    label_slice = label[mid_slice, :, :]
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
    ax[0].imshow(feature_slice, cmap="bone", interpolation="none")
    ax[1].imshow(feature_slice, cmap="bone", interpolation="none")
    ax[1].imshow(label_slice, cmap="prism", vmin=0, vmax=1, alpha=0.5 * (label_slice))
    plt.show()

In [92]:
def set_final_size(image, target_slices, dimension, pad_filler):
    image_size = image.GetSize()
    z = image_size[-1]
    z_diff = target_slices - z
    if z_diff < 0:
        cropped = crop_z(image, np.abs(z_diff))
        return pad_xy(cropped, dimension, pad_filler)
    elif z_diff > 0:
        padded = pad_z(image, np.abs(z_diff), pad_filler)
        return pad_xy(padded, dimension, pad_filler)

    return pad_xy(image, dimension, pad_filler)


def pad_z(image, z_diff, pad_filler):
    z_top_pad = int(np.ceil(z_diff / 2))
    z_bottom_pad = int(np.floor(z_diff / 2))
    padded_image = sitk.ConstantPad(
        image, (0, 0, z_bottom_pad), (0, 0, z_top_pad), pad_filler
    )
    return padded_image


def crop_z(image, z_diff):
    z_top_crop = int(np.ceil(z_diff / 2))
    z_bottom_crop = int(np.floor(z_diff / 2))
    cropped_image = sitk.Crop(image, (0, 0, z_bottom_crop), (0, 0, z_top_crop))
    return cropped_image


def pad_xy(image, dimension, pad_filler):
    image_size = image.GetSize()
    xy_pad1 = int(np.ceil((dimension - image_size[0]) / 2))  # 256-250=6
    xy_pad2 = int(np.floor((dimension - image_size[0]) / 2))
    padded_image = sitk.ConstantPad(
        image, (xy_pad1, xy_pad1, 0), (xy_pad2, xy_pad2, 0), pad_filler
    )
    return padded_image

In [93]:
# LARGEST SAMPLE
feature_lg_img = get_feature(feature_lg)
feature_lg_sp = resample_spacing(feature_lg_img)
feature_lg_sz = resample_size(feature_lg_sp, 2)
# feature_lg_arr = sitk.GetArrayFromImage(feature_lg_sz)
feature_lg_pd = set_final_size(feature_lg_sz, TARGET_SLICES)
feature_lg_result = feature_lg_pd

label_lg_img = get_label(label_lg)
label_lg_sp = resample_spacing(label_lg_img)
label_lg_sz = resample_size(label_lg_sp, 2)
# label_lg_arr = sitk.GetArrayFromImage(label_lg_sz)
label_lg_pd = set_final_size(label_lg_sz, TARGET_SLICES)
label_lg_result = label_lg_pd

# SMALLEST SAMPLE
feature_sm_img = get_feature(feature_sm)
feature_sm_sp = resample_spacing(feature_sm_img)
feature_sm_sz = resample_size(feature_sm_sp, 2)
# feature_sm_arr = sitk.GetArrayFromImage(feature_sm_sz)
feature_sm_pd = set_final_size(feature_sm_sz, TARGET_SLICES)
feature_sm_result = feature_sm_pd

label_sm_img = get_label(label_sm)
label_sm_sp = resample_spacing(label_sm_img)
label_sm_sz = resample_size(label_sm_sp, 2)
# label_sm_arr = sitk.GetArrayFromImage(label_sm_sz)
label_sm_pd = set_final_size(label_sm_sz, TARGET_SLICES)
label_sm_result = label_sm_pd

In [94]:
print("LARGEST SAMPLE")
print(
    feature_lg_img.GetSize(),
    feature_lg_img.GetSpacing(),
)
print(
    feature_lg_result.GetSize(),
    feature_lg_result.GetSpacing(),
)
print(
    label_lg_result.GetSize(),
    label_lg_result.GetSpacing(),
)
print("SMALLEST SAMPLE")
print(
    feature_sm_img.GetSize(),
    feature_sm_img.GetSpacing(),
)
print(
    feature_sm_result.GetSize(),
    feature_sm_result.GetSpacing(),
)
print(
    label_sm_result.GetSize(),
    label_sm_result.GetSpacing(),
)

LARGEST SAMPLE
(512, 512, 134) (0.9765625, 0.9765625, 3.0)
(256, 256, 192) (2.004016064257028, 2.004016064257028, 2.005)
(256, 256, 192) (2.004016064257028, 2.004016064257028, 2.005)
SMALLEST SAMPLE
(512, 512, 90) (0.9765625, 0.9765625, 3.0)
(256, 256, 192) (2.004016064257028, 2.004016064257028, 2.0074626865671643)
(256, 256, 192) (2.004016064257028, 2.004016064257028, 2.0074626865671643)


In [58]:
v.Execute(feature_sm_result)

The operation couldn’t be completed. Unable to locate a Java Runtime that supports (null).
Please visit http://www.java.com for information on installing Java.

The operation couldn’t be completed. Unable to locate a Java Runtime that supports (null).
Please visit http://www.java.com for information on installing Java.



Reading header
Populating metadata table
Populating core metadata
Populating MetadataStore


In [69]:
writer = sitk.ImageFileWriter()
writer.SetImageIO("NiftiImageIO")
writer.SetFileName("/Users/dutking/LOCAL/AI_uni/radlogix/temp/f_lg.nii")
writer.Execute(feature_lg_result)
writer.SetFileName("/Users/dutking/LOCAL/AI_uni/radlogix/temp/l_lg.nii")
writer.Execute(label_lg_result)
writer.SetFileName("/Users/dutking/LOCAL/AI_uni/radlogix/temp/f_sm.nii")
writer.Execute(feature_sm_result)
writer.SetFileName("/Users/dutking/LOCAL/AI_uni/radlogix/temp/l_sm.nii")
writer.Execute(label_sm_result)